In [2]:
%load_ext autoreload
# Always reload modules so that as you change code in src, it gets loaded
%autoreload 2
%matplotlib inline

import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import pandas as pd
import numpy as np
import random
import math
import EoN
import seaborn as sns
import time

from ctrace.simulation import *
from ctrace.dataset import *
from ctrace.recommender import *
from ctrace.problem_label import *
from ctrace.utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
with open(PROJECT_ROOT / "data" / "SIR_Cache" / "albe.json", 'r') as infile:
        j = json.load(infile)
        (S, I1, I2, R) = (j["S"], j["I1"], j["I2"], j["R"])
        infections = j["infections"]

G = load_graph_cville_labels()
state = InfectionState(G, (S, I1, I2, R), 1350, "none", 0.05, True, 0.8, True, 0.8)
while len(state.SIR.I1) + len(state.SIR.I2):
    state.step(DepRound_fair(state))
    print(len(state.SIR.R))

In [7]:
with open(PROJECT_ROOT / "data" / "SIR_Cache" / "mont.json", 'r') as infile:
        j = json.load(infile)
        (S, I1, I2, R) = (j["S"], j["I1"], j["I2"], j["R"])
        infections = j["infections"]

G = load_graph_montgomery_labels()
G = read_extra_edges(G, 0.15)
state = InfectionState(G, (S, I1, I2, R), 750, "none", 0.05, True, 0.8, True, 0.8)
while len(state.SIR.I1) + len(state.SIR.I2):
    state.step(DepRound_fair(state))
    print(len(state.SIR.R))

341
610
1177
1989
3272
5342
8586
13335
19330
25937
32113
36850
39972
41682
42647
43111
43323
43403
43422
43435
43437
43438
